# Team and Project Information

| Name             | E-mail                     | Photo       |
|------------------|----------------------------|-------------|
| Jason Rudianto   | jasonrudianto@berkeley.edu | <img src=https://dchansew.github.io/w261/Team_6-1_Jason_Rudianto.jpg width="200"> |
| Derrick Chan-Sew | dchansew@berkeley.edu      | <img src=https://dchansew.github.io/w261/Team_6-1_Derrick_Chan-Sew.jpg width="200"> |
| Sean Condon      | scondon@berkeley.edu       | <img src=https://dchansew.github.io/w261/Team_6-1_Sean_Condon.png width="200"> |
| Ryan Chen        | bread12035@berkeley.edu    | <img src=https://dchansew.github.io/w261/Team_6-1_Ryan_Chen.png width="200"> |
| Jessica Huber    | jesshuber@berkeley.edu     | <img src=https://dchansew.github.io/w261/Team_6-1_Jessica_Huber.jpg width="200">|


# Project Abstract

In this notebook, we present a methodology for predicting passenger aircraft departure delays of 15 minutes or more, with the goal of reducing delay impact for the airline industry, leading to less travel inconveniences for passengers, and reduced operational costs for airlines. The primary goals of this project phase are to complete exploratory data analysis, finalize input features for delay prediction, and build out an initial machine learning pipeline for training and evaluating classification algorithms.

With a combination of numerical weather data (precipitation, wind, etc.), flight path categorical data (origin & destination state, aircraft & airport info), and a handful of engineered variables, we achieve a classification performance of 81% with Logistic Regression using a Binary Cross Entropy Loss function and a 0.30 L1 Lasso Regularization parameter. We present the details of our feature extraction, engineering, hyperparameter grid search, time-series cross validation, and include results from other algorithms.

In the next project phase, we will consider more sophisticated ML algorithms before deciding on an optimal algorithm & hyperparameters and presenting its performance.

# Project Description

The data used in the project is a join between Airline On-Time Performance Data (Flights) from the Bureau of Transportation and weather station data sourced from the National Centers for Environmental Information. It represents the weather conditions collected by weather stations within proximity of departing flights at airports and detailed flight information. Detailed data descriptions and dictionaries will be covered in the EDA section. 

Phase 2 focuses on EDA, baseline modeling, model refinement and other experimentation.

<img src="https://dchansew.github.io/w261/Team_6-1_Gantt.png" width="1500">




# EDA

We ran exploratory data analysis on both 3 month and 12 month OTPW datasets. We converted and encoded explanatory features of interest into categorical, numeric and ordinal features. Below is a data dictionary of the key features and some visualizations that aided in feature selection. 

## OTPW - Flights

There are a number of key features that we are interested in analyzing. 
- DEP_DEL is a numerical feature which represents the differences in minutes between the scheduled and actual departure time. This is inclusive of early departures.
- DEP_DEL15 is a binary feature derived from DEP_DEL (and its other derived features) which represents whether the flight has been delayed for more than 15 minutes.
- DAY_OF_WEEK is the day of the week of the scheduled flight. This is represented as an integer but is treated categorically.
- OP_CARRIER_FL_NUM is a categorical feature corresponding to the flight number. 
- CRS_DEP_TIME is a string of the scheduled departure time of the flight.
- ORIGIN_AIRPORT_SEQ_ID is the unique identifier for an airport and is a numerical (but treated as a categorical) feature.
- OP_CARRIER is a categorical string corresponding to the carrier of the flight
- DISTANCE is a numerical feature representing the distance betweent he origin and destination airport in miles.
- ORIGIN_STATE is a string feature representing the state of the origin airport
- DEST_STATE is a string feature representing the state of the destination airport.



## OTPW - Weather

Below are the features selected as independent variables pertaining to weather data:

Ordinal features:
- HourlyAltimeterSetting: Atmospheric pressure reduced to sea level using temperature profile of the “standard” atmosphere. Given in inches of Mercury (in Hg).
- HourlyDewPointTemperature: Dew point temperature (in whole degrees Fahrenheit)
- HourlyDryBulbTemperature: This is the dry-bulb temperature and is commonly used as the standard air temperature reported. It is given here in whole degrees Fahrenheit.
- HourlyPrecipitation: Amount of precipitation in inches to hundredths over the past hour. For certain automated stations, precipitation will be reported at sub-hourly intervals (e.g. every 15 or 20 minutes) as an accumulated amount of all precipitation within the preceding hour. A “T” indicates a trace amount of precipitation.
- HourlyPressureChange: Difference in pressure over the past 3 hours. Prefixed with a “+” or “-” to indicate increase or decrease in pressure respectively. Given in inches of Mercury (in Hg).
- HourlyPressureTendency: Pressure tendency (In general a 0 through 3 here indicates an increase in pressure over previous 3 hours and a 5 through 8 indicates a decrease over the previous 3 hours and 4 indicates no change during the previous 3 hours)
- HourlyRelativeHumidity: This is the relative humidity given to the nearest whole percentage.
- HourlySeaLevelPressure: Sea level pressure given in inches of Mercury (in Hg).
- HourlyStationPressure: Atmospheric pressure observed at the station during the time of observation. Given in inches of Mercury (in Hg).
- HourlyVisibility: The horizontal distance an object can be seen and identified given in whole miles. Note visibilities less than 3 miles are usually given in smaller increments (e.g. 2.5).
- HourlyWetBulbTemperature: This is the wet-bulb temperature. It is given here in whole degrees Fahrenheit.
- HourlyWindDirection: Wind direction from true north using compass directions (e.g. 360 = true north, 180 = south, 270 = west, etc.). Note: A direction of “000” is given for calm winds.
- HourlyWindGustSpeed: Wind gusts occurring during time of observation. Given in miles per hour (mph).
- HourlyWindSpeed: Speed of the wind at the time of observation given in miles per hour (mph).
- bad_weather_frequency: Frequency of bad weather events as a percentage, grouped by airport.

One-hot encoded features:
- HourlySkyConditions: one-hot encoded feature that contains 3 character abbreviation of sky conditions. 
  - CLR (clear sky)
  - FEW (few clouds)
  - SCT (scattered clouds)
  - BKN (broken clouds)
  - OVC (overcast)
  - VV (obscured sky)
  - 10 (partially obscured sky).
- HourlyPresentWeatherType: one hot encoded feature that contains 2 character abbreviation of weather type.
  - DZ:01 - Drizzle
  - RA:02 - Rain
  - SN:03 - Snow
  - SG:04 - Snow Grains
  - IC:05 - Ice Crystals
  - PL:06 - Ice Pellets
  - GR:07 - Hail
  - GS:08- Small Hail an/or Snow Pellets
  - UP:09 - Unknown Precipitation
  - BR:1 - Mist
  - FG:2 - Fog
  - FU:3 - Smoke
  - VA:4 - Volcanic Ash
  - DU:5 - Widespread Dust
  - SA:6 - Sand
  - HZ:7 - Haze
  - PY:8 - Spray
  - PO:1 - Well developed dust/sand whirls
  - SQ:2 - Squalls
  - FC:3 - Funnel CLoud, Waterspout or Tornado
  - SS:4 - Sandstorm
  - DS:5 - Duststorm

Reference: https://www.ncei.noaa.gov/products/land-based-station/local-climatological-data

## Correlation Matrices

Using DEP_DEL15 as a target variable, we ran feature and feature/target correlation matrices on the 3 month dataset to inform key independent features: 
 

<img src="https://dchansew.github.io/w261/Phase 2 - Feature Correlation Matrix.png" width="1000">



<img src="https://dchansew.github.io/w261/Phase 2 - Target Correlation Matrix.png" width="1000">

We dropped features with high collinearity and features that we determined would not be available as a predictor such as arrival delays and total elapsed flight time. This information leakage will be discussed further in the following sections. 

Additionally, we used log transformations to scale our input and target data. Below is the log transformation of our target variable, departure delay:

<img src="https://dchansew.github.io/w261/Phase 2 - Log of Departure Delay.png" width="500">


## Summary Statistics

Below is a sample of summary statistics on the 12 month dataset after EDA was performed. This gives us a general understanding of centrality for key features.

<img src="https://dchansew.github.io/w261/Phase 2 - Summary Statistics Sample.png" width="1500">


# Modeling Pipelines
## EDA, Feature selection, and Feature Engineering
OPTW data was pre-joined with flight data and weather data with an initial feature number of 216 and over 5 million rows. In the EDA section, we plotted descriptive statistics for numerical data and created graphical representations to understand the data better and then we dropped irrelevant features based on the observation we got in each data set. Secondly, we categorized our features into categorical, numeric, and ordinal features and treated them separately. Detailed methods are described in the EDA section. Next, we plotted the correlation matrix table to figure out if there is any multicollinearity between features that may potentially cause overfitting in the training process and to check out the correlation between features and the target variable and make it the basis for dimension reduction. Lastly, we delve into features to check the distribution of the data, if it is severely skewed then it may need to take the log transformation, or if the data is imbalanced, we may sample the data based on the final performance of our model. To better predict our target variable, we create two frequency-based features: the frequency of flight delays by airports and the frequency of bad weather happening in each airport. These two features can help us better train the model. We saved EDA-processed data to the blob storage and retrieved it for spark pipeline building. 

## PySpark Pipeline Creation

<img src="https://dchansew.github.io/w261/Phase 2 - Pipeline.PNG" width="1000">

In building the spark pipeline for further training, We only need to take normalization on ordinal and numeric features, which is beneficial to model training with a fixed learning rate. In other words, the model can learn the pattern more evenly from each ordinal and numeric features. Secondly, we encapsulate the logistic regression model as our baseline model and the stage of the pipeline goes numeric_assembler, scaler, final_assembler, and lr where numeric_asseumbler takes numeric and ordinal features as a family and sends them to scaler for further normalization. Next, the final_asseumbler would append the one-hot encoded features and pass them down to the logistic regression model. The hyperparameters of the logistic regression model here are elasticNetParam which means the ratio of L1 and L2  and regParam which means L1 respectively, and we would find the best set in the grid search stage. As for the batch size, loss function, and optimizer, the default settings of the logistic regression model in Pyspark are full batch training, binary cross-entropy, and L-BFGS respectively.

Binary Cross-Entropy Loss Function with L1 and L2 Regularization:
$$ L = -\frac{1}{N} \sum_{i=1}^{N} [y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)] + \lambda \sum_{j=1}^{m} |w_j| + \lambda \sum_{j=1}^{m} w_j^2 $$




## Data Splitting and Time-Series Cross-Validation
Firstly, we split our data into training and testing with portions of 90% and 10%. As for validation data, We would further divide training data into training and validation data in the customized time-series-cross-validation function. In the stage of cross-validation, since our weather data is seasonal, it's inappropriate to validate summer data with winter data as a conventional cross-validation method. Here, we applied the rolling cross-validation method and split the 12 months' data into 12 folds. In the first iteration, training with data of Jan. and validating with data of Feb. Next, training with data of Jan. and Feb. and validating with data of Mar. and so on. In the last iteration, we would train the model with data from Jan. to Nov. and validate it using Dec. Since we only have data for 12 months, the data of Dec. would not be fed to train the model. 

## Grid Search for Hyperparameter of Regularization 
The hyperparameters of the logistic regression model for the regularization here are elasticNetParam which means the ratio of L1 and L2 and regParam which means L1. In terms of purpose, L1 would encourage the sparsity of the data by adding an L1-times-weights term to the loss function and L2 would discourage the large terms of the data by adding an L2-times-square-weights term to the loss function. The initial setup for elasticNetParam and regParam are 0.8 and 0.3 respectively and we end up getting a training and validation accuracy both around 80% and no overfitting spotted. Through the grid search process, we noticed that the best pair of hyperparameters is (1, 0.3) which implies no L2 regularization is needed. It's reasonable since we took the normalization on ordinal and numeric features and there are no extremely large features in the first place.

## Experiments
Initially, We have a family of ordinal and numeric features with 38 features, and a family of one-hot encoded features with 164 features, so we have 202 features in total and 5,811,854 rows as our training data. With the initial hyperparameter setup (0.8, 0.3), we have 80%, 81%, and 81% in training, validation, and testing accuracy, and no overfitting was spotted. Surprisingly, if we train the best hyperparameter pair from the grid search we got, the training accuracy surged up to 98% which is way too high and suspicious. As a result, we circled back and found that some features might potentially have information leakage issues. For example, features related to arriving like ARR_DELAY were post-departure data that we would not know before the flight took off and it belongs to future information at the time. 

After we removed features that caused the information leakage, we have 22 features in the ordinal and numeric family which are DAY_OF_WEEK, OP_CARRIER_FL_NUM, ORIGIN_AIRPORT_SEQ_ID, CRS_DEP_TIME, 14 Hourly Weather features, 2 frequency-based features, and TAIL_NUM_delay. As for the one-hot encoded family, we have 154 features which are one-hot encoded OP_CARRIER, origin_type, ORIGIN_STATE_ABR, DEST_STATE_ABR, HourlySkyConditions, and HourlyPresentWeatherType. With the best pair of hyperparameters got in the grid search. we have 80%, 82%, and 81% in training, validation, and testing accuracy, and no overfitting was spotted. Computational-wise, It takes around 27 minutes to fit in the model using Standard-D4ds-16GB-4cores and during the process, the worker number fluctuated in the range of 4 to 6.


<img src="https://dchansew.github.io/w261/Phase 2 - Experiement_table.PNG" width="1000">


# Results 

Our inital baseline experiment was a logistic regression with regularization hyperparameters at 0.8 and 0.3.  We achieved a training accuracy of 80% and test accuracy of 81% which given 82% of the dataset is not a delay longer than 15 minutes we would get the same performance if we had a model that guessed not delayed for every row.  We moved to a grid search to help fine-tune the hyperparamters but achieved a surprising train and test accuracy of 98% an 96% respectively.  We realized we had information leakage and adjusted our data pipeline to remove columns that were futuristic datapoints.  With the fixed leakage issue, we utilized the hyperparamters from the grid search for our final baseline model. While we did not see improvement in our accuracy we did see a jump in precision and recall from 64% to 74% and 76% to 80%, respectively.  This means our model is making fewer false predictions of delays more than 15 minutes and the model is better capturing the actual more than 15 minute delays.  The precision is slightly lower than accuracy and recall, which means for a prediction of delay being made, there may be some misprediction and the model tends to predict delay over not delay. It may be rooted in the imbalanced data of our target variable, but the difference is slim so we will further study it in our next model.

# Conclusion

The financial and mental repercussions of airline travel delays is an important pain point for airline carriers, airports and travelers. With the rich amount of available airline travel data, an ML pipeline can be constructed and trained which can forecast significant departure delays for a given flight – a highly performant model can assist airline carrier and airports to better manage flight schedules and destinations and reduce risk of travel delays. Through iteration and testing, we identified a handful of airline carrier controllable features and non-controllable features that can be used to create a logistic regression model that achieved an accuracy of 81%. With a highly accurate model, airline carriers can better identify better flight schedules and conduct cost-benefit analysis on existing offered flights on their delay risk and whether they should continue offering that flight path. Immediate future iterations of this project can explore running additional, more sophisticated graph algorithms, running on the larger 5-year data set and tinkering with different train/test ratios. In the longer term, it would be fruitful to explore more advanced, sophisticated algorithms and techniques, such as random forests, decisions trees, gradient boosting, neural networks and more as there may exist non-linear variable relationships that may affect flight delays. 



# Updated Credit assignment plan

| Name             | Workload                                                                                                                   |
|------------------|----------------------------------------------------------------------------------------------------------------------------|
| Jason Rudianto   | EDA - Flights, Model Pipeline, Presentation                                                                                |
| Derrick Chan-Sew | Correlation, Presentation, Model Pipeline, Visualizations                                                                  |
| Sean Condon      | EDA - Weather, Abstract, Model Pipeline, Presentation                                                                      |
| Ryan Chen        | EDA - Stations, checkpoint, Model Pipeline, Cross-Validation, GridSearch, Presentation                                     |
| Jessica Huber    | Model Pipeline, Logistic/Linear Regression Baseline, Presentation                                                          |




# Reference

Notebooks:
- https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/2798664673128977/command/2798664673128981
- https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/2798664673111973/command/2798664673111974
- https://adb-4248444930383559.19.azuredatabricks.net/?o=4248444930383559#notebook/2798664673128951/command/2798664673128952